In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, fftshift, ifft2
from PIL import Image
import urllib.request
import io

# Создаем тестовое изображение с различными деталями
def create_test_image(size=256):
    """Создание синтетического изображения с разными типами деталей"""
    image = np.zeros((size, size))
    
    # Крупные объекты (низкие частоты)
    image[50:150, 50:150] = 0.8  # Большой квадрат
    image[180:230, 30:80] = 0.6   # Средний квадрат
    
    # Мелкие детали (высокие частоты)
    # Тонкие линии
    for i in range(size):
        if i % 20 == 0:
            image[i, :] = 0.3  # Горизонтальные линии
            image[:, i] = 0.3  # Вертикальные линии
    
    # Точки
    for i in range(20, size, 25):
        for j in range(20, size, 25):
            image[i:i+2, j:j+2] = 0.9
    
    # Текст (сложные высокочастотные компоненты)
    image[200:210, 150:200] = 0.7
    image[205:207, 150:200] = 0.9
    
    return np.clip(image, 0, 1)

def fft_compression(image, compression_ratio=0.1):
    """
    Сжатие изображения с помощью FFT
    compression_ratio: доля сохраняемых коэффициентов (0.1 = 10%)
    """
    # Прямое преобразование Фурье
    fft_data = fft2(image)
    fft_shifted = fftshift(fft_data)
    
    # Амплитудный спектр для анализа
    magnitude = np.abs(fft_shifted)
    
    # Определяем порог отсечения
    total_coeffs = image.shape[0] * image.shape[1]
    target_coeffs = int(total_coeffs * compression_ratio)
    
    # Находим пороговое значение амплитуды
    sorted_magnitudes = np.sort(magnitude.flatten())[::-1]
    threshold = sorted_magnitudes[target_coeffs - 1] if target_coeffs > 0 else sorted_magnitudes[0]
    
    # Создаем маску для сохранения только самых важных коэффициентов
    mask = magnitude >= threshold
    compressed_fft = fft_shifted * mask
    
    # Обратное преобразование
    compressed_image = np.real(ifft2(fftshift(compressed_fft)))
    
    # Расчет степени сжатия
    original_size = total_coeffs * 2  # действительная + мнимая части
    compressed_size = np.sum(mask) * 2
    actual_compression_ratio = compressed_size / original_size
    
    return compressed_image, mask, actual_compression_ratio

def progressive_compression_demo(image, ratios=[0.001, 0.01, 0.05, 0.1, 0.3]):
    """Демонстрация прогрессивного сжатия с разными коэффициентами"""
    
    plt.figure(figsize=(20, 12))
    
    # Исходное изображение
    plt.subplot(2, 3, 1)
    plt.imshow(image, cmap='gray', vmin=0, vmax=1)
    plt.title('Исходное изображение\n(100% коэффициентов)')
    plt.axis('off')
    
    for i, ratio in enumerate(ratios):
        # Сжатие
        compressed_img, mask, actual_ratio = fft_compression(image, ratio)
        
        # Визуализация результатов
        plt.subplot(2, 3, i+2)
        plt.imshow(compressed_img, cmap='gray', vmin=0, vmax=1)
        plt.title(f'Сжатие: {ratio*100:.1f}% коэффициентов\n'
                 f'Фактическое сжатие: {actual_ratio*100:.1f}%')
        plt.axis('off')
        
        # Расчет PSNR (Peak Signal-to-Noise Ratio)
        mse = np.mean((image - compressed_img) ** 2)
        if mse == 0:
            psnr = 100
        else:
            psnr = 20 * np.log10(1.0 / np.sqrt(mse))
        
        print(f"Коэффициент сжатия: {ratio*100:5.1f}% -> "
              f"PSNR: {psnr:5.2f} dB, "
              f"Сохранено коэффициентов: {np.sum(mask)}")

    plt.tight_layout()
    plt.show()

# Демонстрация с синтетическим изображением
print("=== ДЕМОНСТРАЦИЯ СЖАТИЯ СИНТЕТИЧЕСКОГО ИЗОБРАЖЕНИЯ ===")
test_image = create_test_image(256)
progressive_compression_demo(test_image)

# Демонстрация частотных масок для разных уровней сжатия
def show_compression_masks(image):
    """Визуализация масок сжатия в частотной области"""
    
    ratios = [0.001, 0.01, 0.05, 0.1]
    
    plt.figure(figsize=(16, 4))
    
    for i, ratio in enumerate(ratios):
        fft_data = fft2(image)
        fft_shifted = fftshift(fft_data)
        magnitude = np.abs(fft_shifted)
        
        # Создаем маску
        total_coeffs = image.shape[0] * image.shape[1]
        target_coeffs = int(total_coeffs * ratio)
        sorted_magnitudes = np.sort(magnitude.flatten())[::-1]
        threshold = sorted_magnitudes[target_coeffs - 1] if target_coeffs > 0 else 0
        mask = magnitude >= threshold
        
        plt.subplot(1, 4, i+1)
        plt.imshow(mask, cmap='gray')
        plt.title(f'Маска {ratio*100:.1f}% коэффициентов\n'
                 f'Белые = сохраняются, Черные = отбрасываются')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

print("\n=== ВИЗУАЛИЗАЦИЯ ЧАСТОТНЫХ МАСОК ===")
show_compression_masks(test_image)

# Работа с реальным изображением
def load_real_image():
    """Загрузка реального изображения для демонстрации"""
    try:
        # Попробуем загрузить тестовое изображение
        from skimage import data
        real_image = data.camera().astype(float) / 255.0
        return real_image
    except:
        # Создаем более сложное синтетическое изображение
        print("Используем сложное синтетическое изображение")
        complex_image = create_test_image(300)
        
        # Добавляем градиенты и текстуры
        x, y = np.meshgrid(np.linspace(0, 1, 300), np.linspace(0, 1, 300))
        gradient = 0.3 * (x + y) / 2
        complex_image = np.clip(complex_image + gradient, 0, 1)
        
        return complex_image

print("\n=== ДЕМОНСТРАЦИЯ СЖАТИЯ РЕАЛЬНОГО ИЗОБРАЖЕНИЯ ===")
real_image = load_real_image()
progressive_compression_demo(real_image, ratios=[0.001, 0.005, 0.02, 0.08, 0.2])

# Анализ качества сжатия
def analyze_compression_quality(image):
    """Детальный анализ качества при разных уровнях сжатия"""
    
    compression_ratios = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5]
    psnr_values = []
    mse_values = []
    
    plt.figure(figsize=(15, 5))
    
    for i, ratio in enumerate(compression_ratios):
        compressed_img, mask, actual_ratio = fft_compression(image, ratio)
        
        # Расчет метрик качества
        mse = np.mean((image - compressed_img) ** 2)
        if mse == 0:
            psnr = 100
        else:
            psnr = 20 * np.log10(1.0 / np.sqrt(mse))
        
        psnr_values.append(psnr)
        mse_values.append(mse)
        
        # Визуализация для некоторых уровней сжатия
        if ratio in [0.001, 0.01, 0.1]:
            idx = [0.001, 0.01, 0.1].index(ratio) + 1
            plt.subplot(1, 3, idx)
            plt.imshow(compressed_img, cmap='gray')
            plt.title(f'{ratio*100:.1f}% коэф.\nPSNR: {psnr:.1f} dB')
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # График зависимости качества от степени сжатия
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.semilogx(compression_ratios, psnr_values, 'bo-', linewidth=2)
    plt.xlabel('Доля сохраняемых коэффициентов')
    plt.ylabel('PSNR (dB)')
    plt.title('Качество vs Сжатие')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.loglog(compression_ratios, mse_values, 'ro-', linewidth=2)
    plt.xlabel('Доля сохраняемых коэффициентов')
    plt.ylabel('MSE')
    plt.title('Ошибка vs Сжатие')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("\n=== АНАЛИЗ КАЧЕСТВА СЖАТИЯ ===")
analyze_compression_quality(test_image)